## Chunking and Storing Extrated Data

All the extracted csv in /output will be semantically chunked and stored in the QDrant collection "owners_manual_chunks" along with the metadata, which has all the details about the chunk.

In [1]:
%%capture
!pip install langchain
!pip install langchain_openai langchain_experimental
!pip install langchainhub
!pip install fastembed
!pip install qdrant-client
!pip install langchain-cohere
!pip install einops
!pip install PyPDF2
!pip install pypdf
!pip install -U langchain-qdrant
!pip install transformers -U

In [2]:
import os
os.environ["TOGETHER_API_KEY"] = "148521c4088ad416dced465cc144671626b00c860af4e6ebc855953567087d8a"
os.environ["COHERE_API_KEY"] = 'SmY5KbyyMKH4rLQg9Tn68UgZALRdNdhBoYkBgEF5'

TOP_K = 5
MAX_DOCS_FOR_CONTEXT = 5

In [3]:
from operator import itemgetter
from langchain.vectorstores import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http import models
from langchain_core.documents.base import Document
from langchain.load import dumps, loads
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.retrievers import BaseRetriever
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings
from langchain_cohere.llms import Cohere
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
import PyPDF2
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_qdrant import Qdrant
from itertools import chain

In [5]:
embedding_model = CohereEmbeddings(model = "embed-english-v3.0")
semantic_chunker_embed_model = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [6]:
qdrant_client = QdrantClient(
    "https://35ebdc7d-ec99-4ebd-896c-ff5705cf369b.us-east4-0.gcp.cloud.qdrant.io:6333",
    prefer_grpc=True,
    api_key="9dKJsKOYwT0vGlWPrZXBSIlbUzvRdJ1XkM0_floo8FmYCOHX_Y0y-Q",
)

QDRANT_URL = "https://35ebdc7d-ec99-4ebd-896c-ff5705cf369b.us-east4-0.gcp.cloud.qdrant.io:6333"
QDRANT_API_KEY = "9dKJsKOYwT0vGlWPrZXBSIlbUzvRdJ1XkM0_floo8FmYCOHX_Y0y-Q"

In [34]:
def create_QDrant_collection():
		embeddings = CohereEmbeddings(model = "embed-english-v3.0")
		qdrant_client.recreate_collection(
		collection_name="owners_manual_test",
		vectors_config = models.VectorParams(size=1024, distance=models.Distance.COSINE),
	)

create_QDrant_collection()

/tmp/ipykernel_34/1163320295.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


In [8]:
import cohere

model="embed-english-v3.0"
input_type="search_query"
co = cohere.Client(api_key="SmY5KbyyMKH4rLQg9Tn68UgZALRdNdhBoYkBgEF5")

In [9]:
import pandas as pd
import numpy as np
exter_data = pd.read_csv("/kaggle/input/extracted-image-data/data_hyundai_exter.csv")
exter_data = exter_data[:50]
exter_data.head()

,page_no,page_data,tab_data,image_data
0,0,Exter OWNER'S MANUAL Operation Maintenance Spe...,NaN,NaN
1,1,CAUTION: MODIFICATIONS TO YOUR HYUNDAI Your HY...,NaN,NaN
2,2,Table of Contents Hyundai Warranty Policy Vehi...,<tab>The table contains information regarding ...,NaN
3,3,FOREWORD Thank you for choosing HYUNDAI. We ar...,NaN,NaN
4,4,FUEL REQUIREMENTS Petrol engine Unleaded Your ...,NaN,NaN


In [10]:
import re
def extract_image_filenames(img_data):
    pattern = r'file_path=\((.*?)\)'
    filenames = re.findall(pattern, img_data)
    return filenames

def extract_image_contents(image_data):
    pattern = r'<img file_path=\(.*?\)>(.*?)</img>'
    image_contents = re.findall(pattern, image_data, re.DOTALL)
    return image_contents

def extract_table_contents(table_data):
    pattern = r'<tab>(.*?)</tab>'
    table_contents = re.findall(pattern, table_data, re.DOTALL)
    return table_contents

In [33]:
def upload_chunks_to_QDrant(doc_type, image_ids, documents, metadata, start_idx):
    records_to_upload = []
    
    vectors = co.embed(texts=documents,
                    model=model,
                    input_type=input_type)
    
    for idx, chunk in enumerate(documents):
        chunk_metadata = metadata.copy()
        chunk_metadata["chunk_type"] = doc_type
        chunk_metadata["image_ids"] = image_ids[idx] if doc_type == "Image" else image_ids
        content = f"This {doc_type} is extracted from manual of {car_name}. {chunk}"
        payload = {"page_content": content, "metadata": chunk_metadata}
        
        record = models.PointStruct(
            id=start_idx + idx,
            vector=vectors.embeddings[idx],
            payload=payload
        )
        records_to_upload.append(record)

    qdrant_client.upload_points(
        collection_name="owners_manual_test",
        points=records_to_upload
    )

In [30]:
def upload_pdf_to_QDrant(extracted_car_data, car_name, start_idx=0):
    semantic_chunker = SemanticChunker(
        semantic_chunker_embed_model, breakpoint_threshold_type="percentile"
    )

    current_idx = start_idx
    for _, row in extracted_car_data.iterrows():
        page_num = row['page_no']
        page_data = row['page_data']
        table_data = row['tab_data']
        image_data = row['image_data']
        
        image_ids = None if image_data is np.nan else extract_image_filenames(image_data)
        
        metadata = {"car_name":car_name,
                    "page_number": page_num,
                    }

        #Insert Text
        if page_data is not np.nan:
            documents = semantic_chunker.create_documents([page_data])

            text_content = []
            for doc in range(len(documents)):
                text_content.append(documents[doc].page_content)

            upload_chunks_to_QDrant("Text", image_ids, text_content, metadata, current_idx)
            current_idx += len(documents)
        
        # Insert Images
        if image_data is not np.nan:
            image_content = extract_image_contents(image_data)
            upload_chunks_to_QDrant("Image", image_ids, image_content, metadata, current_idx)
            current_idx += len(image_content)
            
        #Insert Tables
        if table_data is not np.nan:
            table_content = extract_table_contents(table_data)
            upload_chunks_to_QDrant("Table", image_ids, table_content, metadata, current_idx)
            current_idx += len(table_content)
            
        print(f"chunked page {page_num}")

    return current_idx

In [31]:
csv_files = [
#     ("Tata Nexon", "/kaggle/input/extracted-image-data/nexon_extracted_data.csv"),
     ("Hyundai Exter", "/kaggle/input/extracted-image-data/data_hyundai_exter.csv"),
#     ("Tata Punch", "/kaggle/input/extracted-image-data/punch_extracted_data.csv"),
#     ("Hyundai Verna", "/kaggle/input/extracted-image-data/verna_extracted_data.csv")
]

In [35]:
start_idx = 0
for car_name, csv_path in csv_files:
    manual_df = pd.read_csv(csv_path)
    start_idx = upload_pdf_to_QDrant(manual_df[:50], car_name, start_idx)

chunked page 0
chunked page 1
chunked page 2
chunked page 3
chunked page 4
chunked page 5
chunked page 6
chunked page 7
chunked page 8
chunked page 9
chunked page 10
chunked page 11
chunked page 12
chunked page 13
chunked page 14
chunked page 15
chunked page 16
chunked page 17
chunked page 18
chunked page 19
chunked page 20
chunked page 21
chunked page 22
chunked page 23
chunked page 24
chunked page 25
chunked page 26
chunked page 27
chunked page 28
chunked page 29
chunked page 30
chunked page 31
chunked page 32
chunked page 33
chunked page 34
chunked page 35
chunked page 36
chunked page 37
chunked page 38
chunked page 39
chunked page 40
chunked page 41
chunked page 42
chunked page 43
chunked page 44
chunked page 45
chunked page 46
chunked page 47
chunked page 48
chunked page 49


In [5]:
template = """Please answer the [question] using only the following [information]. If there is no [information] available to answer the question, do not force an answer.

Information: {context}

Question: {question}
Final answer:"""

In [7]:
def reciprocal_rank_fusion(results: list[list], k=60):
    """Rerank docs (Reciprocal Rank Fusion)

    Args:
        results (list[list]): retrieved documents
        k (int, optional): parameter k for RRF. Defaults to 60.

    Returns:
        ranked_results: list of documents reranked by RRF
    """

    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # return only documents
    return [x[0] for x in reranked_results[:MAX_DOCS_FOR_CONTEXT]]

In [8]:
def query_generator(original_query: dict) -> list[str]:
    """Generate queries from original query

    Args:
        query (dict): original query

    Returns:
        list[str]: list of generated queries
    """

    # original query
    query = original_query.get("query")

    # prompt for query generator
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful assistant that generates multiple search queries based on a single input query."),
        ("user", "Generate multiple search queries related to:  {original_query}. When creating queries, please refine or add closely related contextual information, without significantly altering the original query's meaning"),
        ("user", "OUTPUT (3 queries):")
    ])

    # LLM model
    model = Cohere()

    # query generator chain
    query_generator_chain = (
        prompt | model | StrOutputParser() | (lambda x: x.split("\n"))
    )

    # gererate queries
    queries = query_generator_chain.invoke({"original_query": query})

    # add original query
    queries.insert(0, "0. " + query)

    return queries

In [98]:
def rrf_retriever(query: str) -> list[Document]:
    """RRF retriever

    Args:
        query (str): Query string

    Returns:
        list[Document]: retrieved documents
    """

    # Retriever
    embedding = CohereEmbeddings(model = "embed-english-v3.0")


    qdrant = Qdrant(
        client=qdrant_client,
        collection_name="owners_manual_test",
        embeddings=embedding,
    )

    retriever = qdrant.as_retriever()

    # RRF chain
    chain = (
        {"query": itemgetter("query")}
        | RunnableLambda(query_generator)
        | retriever.map()
        | reciprocal_rank_fusion
    )

    # invoke
    result = chain.invoke({"query": query})

    return result

In [95]:
def query(query: str, retriever: BaseRetriever):
    """
    Query with vectordb
    """

    # model
    model = Cohere()

    # prompt
    prompt = PromptTemplate(
        template=template,
        input_variables=["context", "question"],
    )

    # Query chain
    chain = (
        {
            "context": itemgetter("question") | retriever,
            "question": itemgetter("question")
        }
        | RunnablePassthrough.assign(
            context=itemgetter("context")
        )
        | {
            "response": prompt | model | StrOutputParser(),
            "context": itemgetter("context"),
        }
    )

    # execute chain
    result = chain.invoke({"question": query})

    return result

In [101]:
retriever = RunnableLambda(rrf_retriever)
result = query("Tell about MODIFICATIONS TO YOUR HYUNDAI", retriever)
print(result['response'])

image_ids = []
for document in result['context']:
    image_ids.append(document.metadata['image_id'])
print(list(set(list(chain(*image_ids)))))

 According to the manual, Hyundai has reserved the right to make changes at any time without prior notice and without obligation to incorporate such changes so that their policy of continual product improvement may be carried out. 

Certain modifications may also be in violation of regulations established by the Department of Transportation and other government agencies in your country. 
The text then specifies that modifications to your Hyundai should not be modified in any way. Such modifications may adversely affect the performance, safety or durability of your HYUNDAI and may, in addition, violate conditions of the limited warranties covering the vehicle. 

Certain modifications may also be in violation of regulations established by the Department of Transportation and other government agencies in your country. It is possible for an improperly installed/adjusted two-way radio or cellular telephone to adversely affect electronic systems. For this reason, we recommend that you carefu